<a href="https://colab.research.google.com/github/JaimeLopezGarrido/labo2025v/blob/main/src/rpart/290_TareaHogar_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea para el Hogar 02

Esta Tarea para el Hogar 02 se entrega el final de la segunda clase
<br> se espera de usted que intente avanzar con los desafios propuestos y que los traiga terminados para la Clase 03, ya que se analizarán los resultados

##  1. Ensembles de Modelos

Vea el siguiente video [BBC - The Code - The Wisdom of the Crowd](https://www.youtube.com/watch?v=iOucwX7Z1HU)    ( 5 min)


Lea los siguientes artículos


*   [The Wisdom of Crowds (Vox Populi) by Francis Galton](https://www.all-about-psychology.com/the-wisdom-of-crowds.html)  (10 min)
*   [A Gentle Introduction to Ensemble Learning](https://machinelearningmastery.com/what-is-ensemble-learning/)  (10 min)





---



##  2.  Zero2Hero   primera parte
Se han lanzado los primeros fascículos coleccionables llamados "from Zero to Hero" que muy detalladamente, paso a paso enseñan todo lo necesario de R para entender los scripts oficiales de la asignatura.
Están en el repositorio oficial de la asignatura, carpeta  **src/zero2hero**



---



## 3.  Grid Search

Busque en internet el precido significado de los hiperparámetros de la librería **rpart**  que está implementando el algoritmo **CART**  Classification and Regression Trees  propuesto en el año 1984 por Leo Brieman:

*   cp
*   maxdepth
*   minsplit
*   minbucket

Entienda que valores es razonable tome cada hiperparámetro,  en particular profundice en el hiperparámetro  **cp**  y la posibilidad que tome valores negativos.  Es válido consultar a su amigo de *capacidades especiales*  ChatGPT


En las siguientes celdas a un notebook incompleto, un esqueleto de codigo brindado a modo de facilitarle la tarea de codeo y permitir que su valiosa cognición se concentre temas conceptuales de Ciencia de Datos

Modifiquelo agregando loops para que recorra TODOS los hiperparámetros de rpart  < cp, maxdepth, minsplit, minbucket >, y luego póngalo a correr. Recuerde cambiar por SU semilla
Tenga muy presente la granularidad que eligirá para cada hiperparámetro.

### Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



ln: failed to create symbolic link '/content/buckets/b1/labo1': File exists


limpio el ambiente de R

In [1]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,657244,35.2,1454474,77.7,1326133,70.9
Vcells,1220271,9.4,8388608,64.0,1975128,15.1


In [2]:
# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")
if (!require("primes")) install.packages("primes")
require("primes")

Loading required package: data.table

Loading required package: rpart

Loading required package: parallel

Loading required package: primes



Aqui debe poner SU semiila primigenia

In [36]:
#PARAM <- list()
# reemplazar por su primer semilla
#PARAM$semilla_primigenia <- c(100007)
#PARAM$qsemillas <- 1

#PARAM$training_pct <- 70L  # entre  1L y 99L

# elegir SU dataset comentando/ descomentando
#PARAM$dataset_nom <- "~/datasets/dataset_pequeno.csv"


# -------------------------------------------------------------
# CONFIGURACIÓN DE PARÁMETROS Y SEMILLA
# -------------------------------------------------------------

PARAM <- list()

# Defino la semilla principal
PARAM$semilla_primigenia <- 100007

# Si querés usar varias semillas en tus experimentos Montecarlo, las definís acá:
PARAM$semillas <- c(PARAM$semilla_primigenia)
PARAM$qsemillas <- length(PARAM$semillas)

# Porcentaje del dataset para entrenamiento
PARAM$training_pct <- 70L  # entre 1L y 99L

# Dataset a utilizar
PARAM$dataset_nom <- "~/datasets/dataset_pequeno.csv"

# Establezco la semilla activa en R (clave para reproducibilidad)
set.seed(PARAM$semilla_primigenia)

# Verifico qué semilla quedó activa
cat("✅ Semilla activa:", .Random.seed[1], " (esperado:", PARAM$semilla_primigenia, ")\n")

✅ Semilla activa: 10403  (esperado: 100007 )


In [37]:
# particionar agrega una columna llamada fold a un dataset
#  que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30), agrupa=clase_ternaria, seed=semilla)
#   crea una particion 70, 30

particionar <- function(data, division, agrupa = "", campo = "fold", start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(function(x, y) {
    rep(y, x)
  }, division, seq(from = start, length.out = length(division))))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by = agrupa
  ]
}


In [38]:
ArbolEstimarGanancia <- function(semilla, training_pct, param_basicos) {
  # particiono estratificadamente el dataset
  particionar(dataset,
    division = c(training_pct, 100L -training_pct),
    agrupa = "clase_ternaria",
    seed = semilla # aqui se usa SU semilla
  )

  # genero el modelo
  # predecir clase_ternaria a partir del resto
  modelo <- rpart("clase_ternaria ~ .",
    data = dataset[fold == 1], # fold==1  es training,  el 70% de los datos
    xval = 0,
    control = param_basicos
  ) # aqui van los parametros del arbol

  # aplico el modelo a los datos de testing
  prediccion <- predict(modelo, # el modelo que genere recien
    dataset[fold == 2], # fold==2  es testing, el 30% de los datos
    type = "prob"
  ) # type= "prob"  es que devuelva la probabilidad

  # prediccion es una matriz con TRES columnas,
  #  llamadas "BAJA+1", "BAJA+2"  y "CONTINUA"
  # cada columna es el vector de probabilidades


  # calculo la ganancia en testing  qu es fold==2
  ganancia_test <- dataset[
    fold == 2,
    sum(ifelse(prediccion[, "BAJA+2"] > 0.025,
      ifelse(clase_ternaria == "BAJA+2", 117000, -3000),
      0
    ))
  ]

  # escalo la ganancia como si fuera todo el dataset
  ganancia_test_normalizada <- ganancia_test / (( 100 - PARAM$training_pct ) / 100 )

  return(
    c( list("semilla" = semilla),
      param_basicos,
      list( "ganancia_test" = ganancia_test_normalizada )
     )
  )
}


In [39]:
ArbolesMontecarlo <- function(semillas, param_basicos) {

  # la funcion mcmapply  llama a la funcion ArbolEstimarGanancia
  #  tantas veces como valores tenga el vector  PARAM$semillas
  salida <- mcmapply(ArbolEstimarGanancia,
    semillas, # paso el vector de semillas
    MoreArgs = list(PARAM$training_pct, param_basicos), # aqui paso el segundo parametro
    SIMPLIFY = FALSE,
    mc.cores = detectCores()
  )

  return(salida)
}




In [40]:
# carpeta de trabajo
# por favor cambiar numero de experimento si se cambia el loop principal
setwd("/content/buckets/b1/exp")
experimento <- "HT2900"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [41]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# trabajo solo con los datos con clase, es decir 202107
dataset <- dataset[clase_ternaria != ""]

In [42]:

# genero numeros primos
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia) # inicializo
# me quedo con PARAM$qsemillas   semillas
PARAM$semillas <- sample(primos, PARAM$qsemillas )


In [43]:
# genero la data.table donde van los resultados detallados del Grid Search
# un registro para cada combinacion de < semilla, parametros >

if(file.exists("gridsearch_detalle.txt")){
  tb_grid_search_detalle <- fread("gridsearch_detalle.txt")
}else{
  tb_grid_search_detalle <- data.table(
    semilla = integer(),
    cp = numeric(),
    maxdepth = integer(),
   minsplit = integer(),
    minbucket = integer(),
    ganancia_test = numeric()
  )
}

nrow( tb_grid_search_detalle )

[1] 24

Esta es la parte del código que usted debe expandir a TODOS los hiperparámetros de rpart,
<br>ya que actualmente apenas recorre  maxdepth y  minsplit  dejando fijos  cp=-0.5  y minbucket=5

In [1]:
library(data.table)

# -------------------------------------------------------------
# CONFIGURACIÓN INICIAL
# -------------------------------------------------------------

archivo_grid <- "gridsearch_detalle.txt"

# Cargo histórico si existe
if (file.exists(archivo_grid)) {
  tb_grid_search_detalle <- fread(archivo_grid)
  cat("Histórico cargado. Filas previas:", nrow(tb_grid_search_detalle), "\n")
} else {
  tb_grid_search_detalle <- data.table()
  cat("No existe archivo previo. Se creará uno nuevo.\n")
}

iter <- 0
tb_nuevos_resultados <- data.table()

# -------------------------------------------------------------
# GRID SEARCH
# -------------------------------------------------------------

for (vcp in c(-1, -0.5)) {
  for (vmax_depth in c(11, 6)) {
    for (vmin_split in c(650, 1100)) {
      for (vmin_bucket in c(500, 700)) {

        iter <- iter + 1
        cat("Iteración:", iter, "\n")
        flush.console()

        # Defino hiperparámetros actuales
        param_basicos <- list(
          cp = vcp,
          maxdepth = vmax_depth,
          minsplit = vmin_split,
          minbucket = vmin_bucket
        )

        # Llamo a la función Montecarlo
        ganancias <- ArbolesMontecarlo(PARAM$semillas, param_basicos)

        # Si no devuelve nada, sigo
        if (is.null(ganancias) || length(ganancias) == 0) {
          cat("  ⚠️  Sin resultados para esta combinación, se omite.\n")
          next
        }

        # Combino los resultados
        tb_resultados_iter <- rbindlist(ganancias, fill = TRUE)

        # Agrego hiperparámetros
        tb_resultados_iter[, `:=`(
          cp = vcp,
          maxdepth = vmax_depth,
          minsplit = vmin_split,
          minbucket = vmin_bucket
        )]

        # Acumulo
        tb_nuevos_resultados <- rbind(tb_nuevos_resultados, tb_resultados_iter, fill = TRUE)
      }
    }
  }
}

# -------------------------------------------------------------
# GUARDADO FINAL
# -------------------------------------------------------------

if (nrow(tb_nuevos_resultados) > 0) {
  cols_unicas <- c("cp", "maxdepth", "minsplit", "minbucket")

  tb_combinada <- unique(rbind(tb_grid_search_detalle, tb_nuevos_resultados, fill = TRUE), by = cols_unicas)

  fwrite(tb_combinada, file = archivo_grid, sep = "\t")

  cat("\n✅ Guardado completo.\n")
  cat("Filas nuevas:", nrow(tb_nuevos_resultados), "\n")
  cat("Total acumulado:", nrow(tb_combinada), "\n")

  tb_grid_search_detalle <- tb_combinada

} else {
  cat("\n⚠️ No se generaron resultados nuevos. No se guardó nada.\n")
}

No existe archivo previo. Se creará uno nuevo.
Iteración: 1 


ERROR: Error in ArbolesMontecarlo(PARAM$semillas, param_basicos): could not find function "ArbolesMontecarlo"


In [85]:
fwrite( tb_grid_search_detalle,
   file = "gridsearch_detalle.txt",
   sep = "\t"
)

In [86]:
# cantidad de registros de la tabla
nrow(tb_grid_search_detalle)

[1] 64

In [87]:
# muestro la tabla
tb_grid_search_detalle

semilla,cp,maxdepth,minsplit,minbucket,ganancia_test
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
817127,-0.5,10,750,41,44890000
817127,-0.5,6,20,10,46070000
817127,-0.5,6,10,5,48030000
817127,-0.5,6,10,10,46070000
817127,-0.5,8,20,5,43980000
817127,-0.5,8,20,10,44810000
817127,-0.5,8,10,5,44170000
817127,-0.5,8,10,10,44810000
817127,-0.5,7,30,12,43680000


In [88]:
# genero y grabo el resumen
tb_grid_search <- tb_grid_search_detalle[,
  list( "ganancia_mean" = mean(ganancia_test),
    "qty" = .N ),
  list( cp, maxdepth, minsplit, minbucket )
]


In [89]:
# ordeno descendente por ganancia
setorder( tb_grid_search, -ganancia_mean )


In [90]:
# veo los 10 mejores hiperparámetros
tb_grid_search[1:10]

cp,maxdepth,minsplit,minbucket,ganancia_mean,qty
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
-1.0,8,1000,300,49940000,1
-1.0,6,900,200,49700000,1
-1.0,6,1000,200,49700000,1
-1.0,8,1000,200,49660000,1
-0.5,10,300,400,49510000,1
-0.5,10,400,400,49510000,1
-1.0,6,900,300,49510000,1
-1.0,6,1000,300,49510000,1
-0.5,12,600,300,49260000,1


In [91]:
# genero un id a la tabla
tb_grid_search[, id := .I ]

fwrite( tb_grid_search,
  file = "gridsearch.txt",
  sep = "\t"
)


# 4.  Análisis de resultados de Grid Search

La salida de la corrida anterior queda en ~/buckets/b1/exp/HT2900  que corresponde a su Google Drive
<br>HT significa Hyperparameter Tuning
<br>El Grid Search es un método de fuerza bruta de un altísimo costo computacional.
<br>Queremos ver si es posible crear un algoritmo de optimización de hiperparámetros que se ahorre recorrer ciertas porciones muy malas del espacio de búsqueda. Algo del estilo “cada vez que pruebo una combinación de hiperparámetros donde  cp > 1 , la ganancia es muy mala, con lo cual ni vale la pena perder el tiempo explorando en esa region”


<br>Levante el archivo de salida gridsearch.txt  a una planilla tipo Excel y analícelo detenidamente
<br>Ordene por ganancia_mean descendente
<br>
<br>El de mayor ganancia_mean  decimos que es el primero del ranking
En Zulip, correspondiente channel  #Tarea Hogar 02 , topic Analisis Grid Search   intente contestar estas preguntas:

* ¿Qué combinaciones de hiperparámetros poseen una ganancia muy buena?
* ¿Hay algun hiperparámetro que para cierto valor siempre genera una ganancia muy mala, a independientemente de lo que valgan los otros hiperparámetros ?
* ¿Que combinaciones de hiperparámetros es pésima y hubiera sido bueno ahorrarse esas corridas ?

( tiempo estimado 30 minutos, dificultad media )